In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from PIL import Image
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import EarlyStopping


In [ ]:
# !wget -N "https://cainvas-static.s3.amazonaws.com/media/user_data/cainvas-admin/insect_bite.zip"
# !unzip -qo insect_bite.zip
# !rm insect_bite.zip

In [ ]:
data_dir = 'insect bite'

print("Number of samples in - ")
for f in os.listdir(data_dir + '/'):
    if os.path.isdir(data_dir + '/' + f):
        print('\n'+f.upper())
        for fx in os.listdir(data_dir + '/' + f + '/'):
            print(fx, " : ", len(os.listdir(data_dir + '/' + f +'/' + fx + '/')))

In [ ]:
batch = 8

print("Train dataset")
train_ds = tf.keras.preprocessing.image_dataset_from_directory(data_dir+'/train', batch_size=batch)

print("Validation dataset")
val_ds = tf.keras.preprocessing.image_dataset_from_directory(data_dir+'/validation', batch_size=batch)

print("Test dataset")
test_ds = tf.keras.preprocessing.image_dataset_from_directory(data_dir+'/test', batch_size=batch)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
num_samples = 3

for x in class_names:
    plt.figure(figsize=(10, 10))

    filenames = os.listdir(data_dir + '/train/' + x)

    for i in range(num_samples):
        ax = plt.subplot(1, num_samples, i + 1)
        img = Image.open(data_dir + '/train/' + x + '/' + filenames[i])
        plt.imshow(img)
        plt.title(x)
        plt.axis("off")

In [ ]:
print("Looking into the shape of images and labels in one batch\n")  

for image_batch, labels_batch in train_ds:
    input_shape = image_batch[0].shape
    print("Shape of images input for one batch: ", image_batch.shape)
    print("Shape of images labels for one batch: ", labels_batch.shape)
    break

In [ ]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),    # Flip along both axes
        tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),    # Randomly zoom images in dataset
        tf.keras.layers.experimental.preprocessing.RandomRotation((-0.1, 0.1))
    ])


print("Train size (number of batches) before augmentation: ", len(train_ds))

# Apply only to train set    
aug_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

print("Size (number of batches) of augmented dataset: ", len(aug_ds))

#Adding to train_ds
train_ds = train_ds.concatenate(aug_ds)

print("Train size (number of batches) after augmentation: ", len(train_ds))

In [ ]:
base_model = tf.keras.applications.VGG16(weights='imagenet', input_shape=input_shape, include_top=False)    # False, do not include the classification layer of the model

base_model.trainable = False

inputs = tf.keras.Input(shape=input_shape)

x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(len(class_names), activation = 'softmax')(x)    # Add own classififcation layer

model = tf.keras.Model(inputs, outputs)

cb = [EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights = True)]
model.summary()

In [ ]:
model.compile(loss=SparseCategoricalCrossentropy(), optimizer=Adam(0.01), metrics=['accuracy'])

history1 = model.fit(train_ds, validation_data =  val_ds, batch_size = 32, epochs=100, callbacks=cb)

In [ ]:
model.compile(loss=SparseCategoricalCrossentropy(), optimizer=Adam(0.001), metrics=['accuracy'])

history2 = model.fit(train_ds, validation_data =  val_ds, batch_size = 32, epochs=100, callbacks=cb)

In [ ]:
model.evaluate(test_ds)

In [ ]:
def plot(history1, history2, variable1, variable2):
 
    var1_history = history1[variable1]
    var1_history.extend(history2[variable1])
    
    var2_history = history1[variable2]
    var2_history.extend(history2[variable2])
  
    plt.plot(range(len(var1_history)), var1_history)
    plt.plot(range(len(var2_history)), var2_history)
    plt.legend([variable1, variable2])
    plt.title(variable1)

In [ ]:
plot(history1.history, history2.history, "accuracy", 'val_accuracy')


In [ ]:
plot(history1.history, history2.history, "loss", 'val_loss')

In [ ]:
plt.figure(figsize=(20, 20))

for i in test_ds.as_numpy_iterator():
    img, label = i  
    for x in range(len(label)):  
        ax = plt.subplot(1, len(label), x + 1)
        plt.axis('off') 
        plt.imshow(img[x])   
        output = model.predict(np.expand_dims(img[x],0))   
        pred = np.argmax(output[0])   
        t = "Prdicted: " + class_names[pred] 
        t = t + "\nTrue: " + class_names[label[x]]
        t = t + "\nProbability: " + str(output[0][pred])
        plt.title(t)

In [ ]:
model.save('insect.h5')

In [ ]:
import h5py
f = h5py.File("insect.h5",'r')
for item in f.keys():
    print (item)

In [ ]:
f = h5py.File("insect.h5",'r')
for item in f.require_group('model_weights').keys():
    print ('model_weights/' +item)